# Battery Module Structural Analysis (Python)
This notebook analyzes the SES/UTS load cases for battery segments using Python. 

## 1. Constants and Geometry
We define the conversion factors and the physical dimensions of the segment.
$$A_{top} = 2 \cdot (L_{depth} \cdot t_{g})$$
$$A_{width} = 2 \cdot (L_{depth} \cdot t_{g})$$

             ____________________________________________________________
            /                                                           /|
           /                     POLYCARBONATE LID                     / |
          /___________________________________________________________/  |
         |                                                           |   | <--- EneSegHeight
         |                                                           |   |      (105.6mm)
         |___________________________________________________________|   |
         |                                                           |  /
         |                                                           | / <--- EneSegmentWidth
         |___________________________________________________________|/        (81.2mm)
           <------------------- EneSegmentDepth -------------------> 
                                 (417.0mm)

         _________________________________________________________   
      | | [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ] | 
      | | [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ] | 
      | | [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ] |  (81.2mm)WIDTH
      | | [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ]          | 
      | |_________________________________________________________| |
      |                                                             |
        <---------------------- DEPTH (417.0mm) ------------------->

In [ ]:
import math

# Conversion Constants
PSI_TO_PA = 6894.76
MM_TO_M = 1/1000
INCH_TO_M = 25.4/1000

# Module Dimensions
ene_mod_width = 20.3 * MM_TO_M
ene_seg_height = 105.6 * MM_TO_M
ene_mod_depth = 69.5 * MM_TO_M
module_weight = 0.278  # kg

# Segment Assembly
ene_segment_depth = 6 * ene_mod_depth #This represents the longest dimension of the segment
ene_segment_width = ene_mod_width * 4 #This repersents the width fromm 4 enepaq bricks side by side
garolite_thickness = (1/16) * INCH_TO_M
segment_modules_weight = module_weight * 23 #total enepaq mass

# Areas Garolite - Imagine Section Views
top_cross_sec_area = ene_segment_depth * garolite_thickness * 2 #airflow "endplates" arent considerd valid area   # View Axis: Depth X, Width Y
width_cross_sec_area = ene_segment_depth * garolite_thickness * 1 #airflow "endplates" arent considerd valid area # View Axis: Depth X, Height Y
depth_cross_sec_area = (ene_segment_width * garolite_thickness * 2) + (ene_seg_height * garolite_thickness * 2)   # View Axis: Width X, Height Y

Segment Depth Area: 5.930900e-04 m^2


## 2. Load Cases & Bending Stress
Calculated for 20g vertical and 40g lateral loads.
$$\sigma_b = \frac{M \cdot c}{I} \quad \text{where} \quad M = \frac{P \cdot L^2}{12}$$


In [ ]:
# Loading (F = m * a)
g = 9.80665
top_force = segment_modules_weight * 20 * g
width_force = 6 * segment_modules_weight * 40 * g

# Polycarbonate Lid (1/4 inch)
poly_thickness = 0.25 * INCH_TO_M
poly_strength = 62e6
poly_modulus = 2.228e9

# Bending Stress Calculation
m_a = (top_force * (ene_segment_depth**2)) / 12
i_lid = (ene_segment_width * (poly_thickness**3)) / 12
stress_bending_lid = (m_a * (poly_thickness / 2)) / i_lid
sf_lid = poly_strength / stress_bending_lid

print(f"Lid Safety Factor: {sf_lid:.4f}")

## 3. Buckling Analysis (Euler-Johnson)
This section checks if the casing fails via elastic buckling or material yielding.
$$C_c = \pi \sqrt{\frac{2E}{\sigma_y}}$$


In [ ]:
# Garolite Properties
garolite_strength = 38000 * PSI_TO_PA
garolite_modulus = 2400000 * PSI_TO_PA

# Moment of Inertia for the walls
i_walls = 2 * (((ene_segment_width/2 + garolite_thickness/2)**2 * garolite_thickness * (ene_seg_height + (2 * garolite_thickness))))

# Slenderness Ratios
radius_of_gyration = math.sqrt(i_walls / depth_cross_sec_area)
slenderness_ratio = ene_segment_depth / radius_of_gyration
critical_slenderness = math.pi * math.sqrt((2 * garolite_modulus) / garolite_strength)

if slenderness_ratio < critical_slenderness:
    # Johnson Inelastic Buckling
    max_force_buckle = depth_cross_sec_area * (garolite_strength - (garolite_strength**2 * slenderness_ratio**2) / (4 * math.pi**2 * garolite_modulus))
    mode = "Johnson"
else:
    # Euler Elastic Buckling
    max_force_buckle = (math.pi**2 * garolite_modulus * i_walls) / (ene_segment_depth**2)
    mode = "Euler"

sf_buckling = max_force_buckle / width_force

print(f"Buckling Mode: {mode}")
print(f"Buckling Safety Factor: {sf_buckling:.4f}")